<a href="https://colab.research.google.com/github/nsubbaian/FrequentistML/blob/master/Project3/FML_Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Frequentist Machine Learning Assigment 3: Model Assessment and Selection**

Re-implement the example in section 7.10.2 using any simple, out of the box classifier
(like K nearest neighbors from sci-kit). Reproduce the results for the incorrect and 
correct way of doing cross-validation.


In [100]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RepeatedKFold

# Data simulating N = 50 samples in two equal-sized classes, and p = 5000 quantitative
# predictors (standard Gaussian) that are independent of the class labels.
# The true (test) error rate of any classifier is 50%.
X = np.random.normal(0, 1, [50,5000])

Y = np.concatenate([np.zeros(25),np.ones(25)])
np.random.shuffle(Y)

### Results for the incorrect way of doing cross-validation

In [101]:
# Screen the predictors: find a subset of “good” predictors that show fairly strong (univariate) correlation with the class labels
X_new = preprocessing.MinMaxScaler().fit_transform(X)
X_new = SelectKBest(chi2, k=100).fit_transform(X_new, Y)

# Using just this subset of predictors, build a multivariate classifier.
neigh = KNeighborsClassifier(n_neighbors=1)

# Use cross-validation to estimate the unknown tuning parameters and to estimate the prediction error of the final model.
CV = []
kf = RepeatedKFold(n_splits=5, n_repeats=50)

for train_index, test_index in kf.split(X_new):
    X_train, X_test = X_new[train_index], X_new[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    neigh.fit(X_train, Y_train)
    CV.append(1-neigh.score(X_test, Y_test))

print("Average CV Error Rate:", np.array(CV).mean())

Average CV Error Rate: 0.027199999999999995


### Results for the correct way of doing cross-validation

In [102]:
# Divide the samples into K cross-validation folds (groups) at random.

# For each fold k = 1, 2, . . . , K
# 1. Find a subset of “good” predictors that show fairly strong (univariate) correlation with the class labels, using all of the samples except those in fold k.
# 2. Using just this subset of predictors, build a multivariate classifier, using all of the samples except those in fold k.
# 3. Use the classifier to predict the class labels for the samples in fold k.

X = pd.DataFrame(data=X)
Y = pd.DataFrame(data=Y)

CV2 = []
kf = RepeatedKFold(n_splits=5, n_repeats=50)

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]

    X_new2 = preprocessing.MinMaxScaler().fit_transform(X_train)
    X_new2 = pd.DataFrame(data=X_new2)
    selector = SelectKBest(chi2, k=100)
    selector.fit_transform(X_new2, np.ravel(Y_train))
    cols = selector.get_support(indices=True)
    X_new2 = X_new2.iloc[:,cols]

    neigh = KNeighborsClassifier(n_neighbors=1)
    neigh.fit(X_new2, np.ravel(Y_train))
  
    CV2.append(1-neigh.score(X_test.iloc[:,cols], np.ravel(Y_test)))

print("Average CV Error Rate:", np.array(CV2).mean())

Average CV Error Rate: 0.55
